In [1]:
import numpy as np
import matplotlib.pyplot as plt
from termcolor import colored

import tensorflow as tf
import tensorflow_datasets as tfds

from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Flatten,Dense,Activation

from tensorflow.keras.losses import SparseCategoricalCrossentropy
from tensorflow.keras.optimizers import SGD,Adam

from tensorflow.keras.metrics import Mean,SparseCategoricalAccuracy

In [2]:
def get_mnist_ds():
    (train_validation_ds, test_ds) ,ds_info = tfds.load(name='mnist',
                                                        shuffle_files=True,
                                                        as_supervised=True,
                                                        split=['train','test'],
                                                               with_info=True)
    n_train_validation = ds_info.splits['train'].num_examples

    train_ratio = 0.8
    n_train = int(n_train_validation * train_ratio)
    n_validation = n_train_validation -n_train

    train_ds = train_validation_ds.take(n_train)
    remaning_ds = train_validation_ds.skip(n_train)
    validation_ds = remaning_ds.take(n_validation)

    return train_ds,validation_ds,test_ds

train_ds,validation_ds,test_ds=get_mnist_ds()

In [3]:
validation_ds

<TakeDataset shapes: ((28, 28, 1), ()), types: (tf.uint8, tf.int64)>

In [4]:
def standardization(TRAIN_BATCH_SIZE,TEST_BATCH_SIZE):
    global train_ds, validation_ds, test_ds

    def stnd(images, labels):
        images = tf.cast(images, tf.float32) /255.
        return [images, labels]
    
    train_ds = train_ds.map(stnd).shuffle(1000).batch(TRAIN_BATCH_SIZE)
    validation_ds = validation_ds.map(stnd).batch(TEST_BATCH_SIZE)
    test_ds = test_ds.map(stnd).batch(TEST_BATCH_SIZE)



In [5]:
class MNIST_Classfier(Model):
    def __init__(self):
        super(MNIST_Classfier, self).__init__()

        self.flatten = Flatten()
        self.d1 = Dense(64, activation= 'relu')
        self.d2 = Dense(10, activation= 'softmax')
    def call(self,x):
        x = self.flatten(x)
        x= self.d1(x)
        x = self.d2(x)
        return x

In [6]:
def load_metrics():
    global train_loss, train_acc
    global validation_loss, validation_acc
    global test_loss, test_acc

    train_loss = Mean()
    validation_loss = Mean()
    test_loss = Mean()

    train_acc = SparseCategoricalAccuracy()
    validation_acc = SparseCategoricalAccuracy()
    test_acc = SparseCategoricalAccuracy()

In [7]:
@tf.function
def trainer():
    global train_ds, model, loss_object, optimizer
    global train_loss, train_acc
    for images, labels in train_ds:
        with tf.GradientTape() as tape:
            predictions = model(images)
            loss = loss_object(labels, predictions)
        gradients = tape.gradient(loss, model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

        train_loss(loss)
        train_acc(labels, predictions)

In [8]:
@tf.function
def validation():
    global validation_ds, model, loss_object
    global validation_loss, validation_acc
    for images, labels in validation_ds:
        predictions = model(images)
        loss = loss_object(labels, predictions)

        validation_loss(loss)
        validation_acc(labels, predictions)

In [9]:
@tf.function
def tester():
    global test_ds, model, loss_object
    global test_loss, test_acc
    for images, labels in test_ds:
        predictions = model(images)
        loss = loss_object(labels, predictions)

        test_loss(loss)
        test_acc(labels, predictions)
'''
    template ='test Loss: {:.4f}\t test accuracy:{:.2f}%\n'

    print(template.format(test_loss.result(),
                          test_acc.result() * 100))
'''

"\n    template ='test Loss: {:.4f}\t test accuracy:{:.2f}%\n'\n\n    print(template.format(test_loss.result(),\n                          test_acc.result() * 100))\n"

In [10]:
def train_reporter():
    global epoch
    global train_loss, train_acc
    global validation_loss, validation_acc
    #on 
    print(colored('Epoch','red','on_white'), epoch+1)

    template = 'Train Loss: {:.4f}\t Train Accuracy:{:.2f}%\n' +\
    'validation Loss: {:.4f}\t valiation accuracy:{:.2f}%\n'

    print(template.format(train_loss.result(),
                          train_acc.result() * 100,
                          validation_loss.result(),
                          validation_acc.result() * 100))
    train_acc.reset_states()
    train_loss.reset_states()
    validation_loss.reset_states()
    validation_acc.reset_states()

In [11]:
EPOCHS = 10
LR = 0.001
TRAIN_BATCH_SIZE = 16
TEST_BATCH_SIZE = 32

standardization(TRAIN_BATCH_SIZE,TEST_BATCH_SIZE)


In [12]:
model = MNIST_Classfier()
loss_object = SparseCategoricalCrossentropy()
optimizer = SGD(learning_rate=LR)

load_metrics()

for epoch in range(EPOCHS):
    trainer()
    validation()
    train_reporter()

tester()

Epoch 1
Train Loss: 1.4219	 Train Accuracy:66.02%
validation Loss: 0.8731	 valiation accuracy:81.85%

Epoch 2
Train Loss: 0.6993	 Train Accuracy:84.25%
validation Loss: 0.5879	 valiation accuracy:85.82%

Epoch 3
Train Loss: 0.5314	 Train Accuracy:86.72%
validation Loss: 0.4888	 valiation accuracy:87.52%

Epoch 4
Train Loss: 0.4597	 Train Accuracy:88.00%
validation Loss: 0.4377	 valiation accuracy:88.52%

Epoch 5
Train Loss: 0.4190	 Train Accuracy:88.73%
validation Loss: 0.4057	 valiation accuracy:89.12%

Epoch 6
Train Loss: 0.3922	 Train Accuracy:89.29%
validation Loss: 0.3841	 valiation accuracy:89.59%

Epoch 7
Train Loss: 0.3728	 Train Accuracy:89.71%
validation Loss: 0.3685	 valiation accuracy:89.97%

Epoch 8
Train Loss: 0.3579	 Train Accuracy:90.02%
validation Loss: 0.3557	 valiation accuracy:90.34%

Epoch 9
Train Loss: 0.3458	 Train Accuracy:90.35%
validation Loss: 0.3464	 valiation accuracy:90.55%

Epoch 10
Train Loss: 0.3358	 Train Accuracy:90.61%
validation Loss: 0.3372	 valiat